In [31]:
#import libraries
import torch 
import torch.nn as nn 
import numpy as np

In [32]:
#import dataset
import seaborn as sns
iris = sns.load_dataset('iris')

#convert from pd dataframe to tensor
data = torch.tensor( iris[iris.columns[0:4]].values ).float()

#transform species to number
labels = torch.zeros(len(data), dtype=torch.long)

#labels[iris.species=='setosa'] = 0 #Not needed
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

## Separate data into train and test

In [33]:
# No devset here

#How many training examples
propTraining = .8 #in proportion, not percent
nTraining = int(len(labels)*propTraining)

#initialize a boolean vector to select data and labels
traintestBool = np.zeros(len(labels), dtype=bool)

#Is it the correct way to select sample
#traintestBool[range(nTraining)] = True

#this is better, but why?
items2use4train = np.random.choice(range(len(labels)),nTraining, replace=False)
traintestBool[items2use4train] = True

traintestBool


array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True, False,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True, False, False, False, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,

In [34]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [35]:
# Test whether it sbalanced
print('Average of full data:')
print(torch.mean(labels.float()) ) # =1 by definition
print(' ')

print('Average of training data:')
print(torch.mean(labels[traintestBool].float())) #should be 1
print(' ')

print('Average of test data:')
print(torch.mean(labels[~traintestBool].float())) #should also be 1

Average of full data:
tensor(1.)
 
Average of training data:
tensor(1.)
 
Average of test data:
tensor(1.)


In [36]:
# Create the model

#model architecture
ANNiris = nn.Sequential(
    nn.Linear(4,64), #input layer
    nn.ReLU(), #activation unit
    nn.Linear(64,64), #hidden layer
    nn.ReLU(), #activation unit
    nn.Linear(64,3), #output units
)

#loss function
lossfun = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.SGD(ANNiris.parameters(), lr=.01)

In [37]:
# entire dataset
print(data.shape)

#training set
print(data[traintestBool,:].shape )

#test set
print( data[~traintestBool,:].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


## Train and evaluate the model

In [38]:
# train the model
numepochs = 1000

#initialize losses
losses = torch.zeros(numepochs)
ongoingAcc = []

#loop over epochs
for epoch_idx in range(numepochs):

    #forward pass
    yHat = ANNiris(data[traintestBool,:])

    #compute accuracy (note: denser than previous code)
    ongoingAcc.append( 100*torch.mean(
        (torch.argmax(yHat,axis=1) == labels[traintestBool]).float()))

    #compute loss
    loss = lossfun(yHat, labels[traintestBool])
    losses[epoch_idx] = loss 

    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [39]:
# compute train and test accuracies

#final forward pas USING TRAINING DATA
predictions = ANNiris(data[traintestBool,:])
trainacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[traintestBool]).float())

#final forward pass USING TEST DATA
predictions = ANNiris(data[~traintestBool,:])
testacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[~traintestBool]).float())

In [40]:
# report accuracies
print('Final TRAIN accuracy: %g%%' %trainacc)
print('Final TEST accuracy: %g%%' %testacc)


Final TRAIN accuracy: 98.3333%
Final TEST accuracy: 100%
